In [186]:
import torch
import numpy as np
import time

class CustomLogger(object):
    def __init__(self):
        self.losses = []
        self.val_losses = []
        self.deltatime = []
        self.n_epochs = 0
        self.out = []
        
    def update(self, loss, val_loss, delta_time,
               model, loss_fn, train_loader, val_loader):
        self.losses.append(loss)
        self.deltatime.append(delta_time)
        self.val_losses.append(val_loss)
        self.n_epochs += 1

    def output(self):
        s = f'epoch ({self.n_epochs}) ' \
            f'time: {self.deltatime[-1]} ' \
            f'loss: {self.losses[-1]} ' 
        if self.val_losses[-1] is not None:
            s += f'val_loss: {self.val_losses[-1]}'
        self.out.append(s)
        print(s)


class MyLogger(CustomLogger):
    def __init__(self):
        super().__init__()
        self.p = []
    
    def update(self, delta_time, loss, val_loss,
               model, loss_fn, train_loader, val_loader):
        super().update(delta_time, loss, val_loss,
                       model, loss_fn, train_loader, val_loader)
        p = model.state_dict()
        self.p.append([p['linear.bias'].item(), p['linear.weight'].item()])

    def output(self, formatstring=''):
        s = f'epoch ({self.n_epochs}) ' \
            f'time: {self.deltatime[-1]} ' \
            f'loss: {self.losses[-1]} ' \
            f'p: {self.p[-1]}'
        if self.val_losses[-1] is not None:
            s += f'val_loss: {self.val_losses[-1]}' 
        print(s)




In [199]:

class ModelTemplete():
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)

        self.stats = {'losses': [],
                       'val_losses': [],
                       'delta_time': [],
                       'n_epochs': 0}

    def log_update(self, delta_time, loss, val_loss):
        self.stats['delta_time'].append(delta_time)
        self.stats['losses'].append(loss)
        self.stats['val_losses'].append(val_loss)
        self.stats['n_epochs'] += 1

    def log_output(self, verbose=0, formatstr=''):
        s = [f'epoch ({{{formatstr}}})'.format(self.stats['n_epochs'], ),
             f'time: {{{formatstr}}}'.format(self.stats['delta_time'][-1]),
             f'loss: {{{formatstr}}}'.format(self.stats['losses'][-1])]
        if self.stats['val_losses'][-1] is not None:
            s.append(f'val_loss: {{{formatstr}}}'.format(self.stats['val_losses'][-1]))
        if verbose == 1:
            print(' '.join(s))
        return s
    
    def _minibatch_one_epoch(self, dataloader, val=False):
        if val is False:
            self.model.train()
        else:
            self.model.eval()
        # dataloader = self.val_loader if val else self.train_loader

        losses = []
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            yhat = self.model(X_batch)
            loss = self.loss_fn(yhat, y_batch)
            if val is False:
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
            losses.append(loss.item())
        return np.mean(losses)
    
    def train(self, train_loader, val_loader=None, epoch_num=10, verbose=0):
        for _ in range(epoch_num):
            start_time = time.time()
            loss = self._minibatch_one_epoch(train_loader, val=False)
            end_time = time.time()
            delta_time = end_time - start_time

            val_loss = self._minibatch_one_epoch(val_loader, val=True) if val_loader is not None else None
            self.log_update(delta_time, loss, val_loss)
            self.log_output(verbose=verbose)
        # return self.logger



In [182]:
import numpy as np
from torch.nn.modules import Linear
from torch.nn import MSELoss
import torch.nn as nn
from torch.optim import SGD
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


class BetterLR(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.linear = Linear(in_features=1, out_features=1)
        self.linear.bias = torch.nn.Parameter(torch.tensor([1.0], dtype=float))
        self.linear.weight = torch.nn.Parameter(torch.tensor([[1.5]], dtype=float))

    def forward(self, x):
        return self.linear(x)
    

class MyData(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=float).reshape(-1, 1)
        self.y = torch.tensor(y, dtype=float).reshape(-1, 1)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.y)
    
    
RANDOMSEED = 42
np.random.seed(RANDOMSEED)
X = np.random.rand(100)
y = 2.3 + 1.2 * X + np.random.randn(100) * 0.1


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15,
                                                    random_state=RANDOMSEED)
    
train_data = MyData(X_train, y_train)
# val_data = MyData(X_test, y_test)

train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=False)
# val_loader = DataLoader(dataset=val_data, batch_size=32)





In [187]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.2
model = BetterLR().to(device)
optimizer = SGD(model.parameters(), lr=lr)

# msg = MyLogger()
mm = ModelTemplete(model, MSELoss(reduction='mean'), optimizer)


mm.train(train_loader, epoch_num=10, verbose=1)


epoch (1.000) time: 0.003 loss: 0.626
epoch (2.000) time: 0.004 loss: 0.062
epoch (3.000) time: 0.004 loss: 0.046
epoch (4.000) time: 0.003 loss: 0.040
epoch (5.000) time: 0.002 loss: 0.035
epoch (6.000) time: 0.002 loss: 0.031
epoch (7.000) time: 0.004 loss: 0.027
epoch (8.000) time: 0.003 loss: 0.024
epoch (9.000) time: 0.002 loss: 0.022
epoch (10.000) time: 0.002 loss: 0.020


In [201]:
class MyModel(ModelTemplete):
    def __init__(self, model, loss_fn, optimizer):
        super().__init__(model, loss_fn, optimizer)
        self.states['p'] = []

    def log_update(self, delta_time, loss, val_loss):
        super().log_update(delta_time, loss, val_loss)
        p = self.model.state_dict()
        self.states['p'].append([p['linear.bias'].item(), p['linear.weight'].item()])


    def log_output(self, verbose=0):
        s = super().log_output(verbose=0, formatstr=':.6f')
        s.append(f'p: [{self.states['p'][-1][0]}, {self.states['p'][-1][1]}]')
        if verbose==1:
            print(' '.join(s))
        return s

In [202]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.2
model = BetterLR().to(device)
optimizer = SGD(model.parameters(), lr=lr)

# msg = MyLogger()
mm = MyModel(model, MSELoss(reduction='mean'), optimizer)

mm.train(train_loader, epoch_num=10, verbose=1)

epoch (1.000000) time: 0.002618 loss: 0.626386 p: [1.8277028504755573, 1.8368193572906044]
epoch (2.000000) time: 0.002110 loss: 0.061719 p: [1.9607104449826838, 1.8293981130981023]
epoch (3.000000) time: 0.001996 loss: 0.045990 p: [2.001626059409397, 1.7815077539441087]
epoch (4.000000) time: 0.002000 loss: 0.040067 p: [2.0286935704191, 1.7321715193480243]
epoch (5.000000) time: 0.002017 loss: 0.035140 p: [2.0522055690695757, 1.686138097785071]
epoch (6.000000) time: 0.002005 loss: 0.030973 p: [2.0736381185747943, 1.6437403254745735]
epoch (7.000000) time: 0.001977 loss: 0.027453 p: [2.0933134526016604, 1.6047617600958677]
epoch (8.000000) time: 0.002109 loss: 0.024479 p: [2.111393711486754, 1.5689357968513453]
epoch (9.000000) time: 0.001896 loss: 0.021968 p: [2.1280105514943686, 1.5360086358936902]
epoch (10.000000) time: 0.002096 loss: 0.019847 p: [2.143282725696795, 1.505745878510758]


In [109]:
Da = MyData(X, y)
dl = DataLoader(Da)

In [110]:
len(dl)

100

In [152]:




dataset = MyData(X, y)
train_data, val_data = random_split(dataset, [.85, .15], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

In [166]:



lr = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BetterLR().to(device)
optimizer = SGD(model.parameters(), lr=lr)
msg = MyLogger()

mm = ModelTemplete(model, MSELoss(reduction='mean'), optimizer, msg)
mm.set_loaders(train_loader, val_loader)

mm.train(epoch_num=10, verbose=1)    

epoch (1) time: 0.004000 loss: 0.627555 p: [1.828082, 1.851994] 
epoch (2) time: 0.003000 loss: 0.067947 p: [1.972544, 1.843260] 
epoch (3) time: 0.002035 loss: 0.049432 p: [2.016118, 1.792544] 
epoch (4) time: 0.001968 loss: 0.041331 p: [2.036865, 1.736838] 
epoch (5) time: 0.002075 loss: 0.035494 p: [2.055019, 1.686687] 
epoch (6) time: 0.002017 loss: 0.031306 p: [2.071784, 1.641176] 
epoch (7) time: 0.001904 loss: 0.028636 p: [2.099186, 1.604492] 
epoch (8) time: 0.003001 loss: 0.025880 p: [2.116329, 1.565389] 
epoch (9) time: 0.003000 loss: 0.022410 p: [2.119514, 1.524232] 
epoch (10) time: 0.002001 loss: 0.019656 p: [2.137073, 1.495972] 


In [168]:
dir(msg)




['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'deltatime',
 'losses',
 'n_epochs',
 'output',
 'p',
 'update',
 'val_losses']

In [204]:
mm.stats = {"nn": [1,2,2]}



In [206]:
mm.states


{'losses': [0.6263859546406714,
  0.06171929297399448,
  0.045990025048784,
  0.04006740409547522,
  0.03513987757810646,
  0.03097301278504304,
  0.027452771476570853,
  0.02447947611654223,
  0.021968262094932956,
  0.019847355986687194],
 'val_losses': [None, None, None, None, None, None, None, None, None, None],
 'delta_time': [0.002618074417114258,
  0.0021097660064697266,
  0.001996278762817383,
  0.0019998550415039062,
  0.0020172595977783203,
  0.0020046234130859375,
  0.0019769668579101562,
  0.002109050750732422,
  0.0018961429595947266,
  0.002095937728881836],
 'n_epochs': 10,
 'p': [[1.8277028504755573, 1.8368193572906044],
  [1.9607104449826838, 1.8293981130981023],
  [2.001626059409397, 1.7815077539441087],
  [2.0286935704191, 1.7321715193480243],
  [2.0522055690695757, 1.686138097785071],
  [2.0736381185747943, 1.6437403254745735],
  [2.0933134526016604, 1.6047617600958677],
  [2.111393711486754, 1.5689357968513453],
  [2.1280105514943686, 1.5360086358936902],
  [2.1432